In [3]:
import json
import pandas as pd
from nltk.stem import WordNetLemmatizer
import re, unicodedata
from wordcloud import WordCloud, ImageColorGenerator
from nltk.corpus import stopwords
import matplotlib.pyplot as plt
from nltk.tokenize import word_tokenize
import nltk
nltk.download('wordnet')
pd.set_option('display.max_colwidth', -1)
STOPWORDS = set(stopwords.words('english'))

from pyLDAvis import sklearn as sklearn_lda
import pickle 
import pyLDAvis
import os
pyLDAvis.enable_notebook()

[nltk_data] Downloading package wordnet to /Users/jaden/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [4]:
def transform(words):
    contraction_mapping = {"ive":"i have","dont":"do not","cant":"can not","ain't": "is not", "aren't": "are not","can't": "cannot", "'cause": "because", "could've": "could have", "couldn't": "could not", "didn't": "did not",  "doesn't": "does not", "don't": "do not", "hadn't": "had not", "hasn't": "has not", "havent":"have not","haven't": "have not", "he'd": "he would","he'll": "he will", "he's": "he is", "how'd": "how did", "how'd'y": "how do you", "how'll": "how will", "how's": "how is",  "I'd": "I would", "I'd've": "I would have", "I'll": "I will", "I'll've": "I will have","I'm": "I am", "I've": "I have", "i'd": "i would", "i'd've": "i would have", "i'll": "i will",  "i'll've": "i will have","i'm": "i am", "i've": "i have", "isn't": "is not", "it'd": "it would", "it'd've": "it would have", "it'll": "it will", "it'll've": "it will have","it's": "it is", "let's": "let us", "ma'am": "madam", "mayn't": "may not", "might've": "might have","mightn't": "might not","mightn't've": "might not have", "must've": "must have", "mustn't": "must not", "mustn't've": "must not have", "needn't": "need not", "needn't've": "need not have","o'clock": "of the clock", "oughtn't": "ought not", "oughtn't've": "ought not have", "shan't": "shall not", "sha'n't": "shall not", "shan't've": "shall not have", "she'd": "she would", "she'd've": "she would have", "she'll": "she will", "she'll've": "she will have", "she's": "she is", "should've": "should have", "shouldn't": "should not", "shouldn't've": "should not have", "so've": "so have","so's": "so as", "this's": "this is","that'd": "that would", "that'd've": "that would have", "that's": "that is", "there'd": "there would", "there'd've": "there would have", "there's": "there is", "here's": "here is","they'd": "they would", "they'd've": "they would have", "they'll": "they will", "they'll've": "they will have", "they're": "they are", "they've": "they have", "to've": "to have", "wasn't": "was not", "we'd": "we would", "we'd've": "we would have", "we'll": "we will", "we'll've": "we will have", "we're": "we are", "we've": "we have", "weren't": "were not", "what'll": "what will", "what'll've": "what will have", "what're": "what are",  "what's": "what is", "what've": "what have", "when's": "when is", "when've": "when have", "where'd": "where did", "where's": "where is", "where've": "where have", "who'll": "who will", "who'll've": "who will have", "who's": "who is", "who've": "who have", "why's": "why is", "why've": "why have", "will've": "will have", "won't": "will not", "won't've": "will not have", "would've": "would have", "wouldn't": "would not", "wouldn't've": "would not have", "y'all": "you all", "y'all'd": "you all would","y'all'd've": "you all would have","y'all're": "you all are","y'all've": "you all have","you'd": "you would", "you'd've": "you would have", "you'll": "you will", "you'll've": "you will have", "you're": "you are", "you've": "you have" }

    word_list = []
    words = words.replace(".",". ")
    for word in words.split():
        new_word = word
        """Convert all characters to lowercase from list of tokenized words"""
        new_word = new_word.lower()
        """Remove punctuation except for "?" and "!" from list of tokenized words"""
        if contraction_mapping.__contains__(new_word):
            new_word = contraction_mapping[new_word]
        word_list.append(new_word)
    return ' '.join(word_list)

def normalize(words):
    word_list = []
    for word in words:
        """Remove non-ASCII characters from list of tokenized words"""
        new_word = unicodedata.normalize('NFKD', word).encode('ascii', 'ignore').decode('utf-8', 'ignore')
        new_word = re.sub(r"[^\w\s.?!-']", '', new_word)
        lemmatizer = WordNetLemmatizer()
        new_word = lemmatizer.lemmatize(new_word, pos='v')
        if new_word != '':
            word_list.append(new_word)
    return word_list

In [ ]:
data = []
with open("../Crawling/PostProcessing/Menopause_Merged_Discussion.jl") as f:
    df = pd.DataFrame(json.loads(line) for line in f)

In [10]:
def sent_preprocessing(sent):
    return "".join(word + ' ' for word in normalize([word for word in word_tokenize(transform(sent)) if word != None])).strip()

In [11]:
df.info()
df.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8465 entries, 0 to 8464
Data columns (total 5 columns):
content_id    8465 non-null object
post_type     8465 non-null object
poster        8465 non-null object
text          8465 non-null object
timestamp     8465 non-null object
dtypes: object(5)
memory usage: 330.7+ KB


IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.


In [12]:
df["text"] = df.text.apply(lambda x: sent_preprocessing(x))
df.head()

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.


In [13]:
from nltk import ngrams
def get_ngrams(line, n):
    
    result = ''
    token_sent = word_tokenize(line)
    tag_dic = {each[0]:each[1] for each in nltk.pos_tag(token_sent)}
    n_grams = ngrams(token_sent, n)

    """Lemmatize words in list of tokenized words"""
    lemmatizer = WordNetLemmatizer()

    for grams in n_grams:

        condition = False
        if tag_dic[grams[-1]] == "NN" or tag_dic[grams[-1]] == "NNS":
            condition = True
        if set(grams).intersection(STOPWORDS) != set() or set(grams).intersection([',','.','?', '!', ':', '/', '_', '(', ')','...','}', '{', '', '%', 'hour', 'hours','day', 'days', 'week', 'weeks', 'month', 'months','year', 'years', 'hi', 'hey', 'time', 'times', 'today', 'thank', 'thanks','luck', 'morning','night','tonight','mg', 'mgs','', 'weekend', 'wish', 'hope', 'step', 'steps', 'everyone', 'others', 'people']) != set():
            condition = False
        if condition == True:
            phrase = ''
            for each in grams:
                phrase += '_'+ lemmatizer.lemmatize(each, pos='v')
#             if '10_' in phrase:
#                 print(phrase)
            result += ' ' + phrase[1:]
    return result[1:]

In [15]:
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.decomposition import NMF, LatentDirichletAllocation

def display_topics(model, feature_names, no_top_words):
    for topic_idx, topic in enumerate(model.components_):
        print ("Topic %d:" % (topic_idx))
        print (" ".join([feature_names[i]
                        for i in topic.argsort()[:-no_top_words - 1:-1]]))

documents = df.text.apply(lambda x: get_ngrams(x, 2))

no_features = 2000

ct_vectorizer = CountVectorizer(max_df=0.95, min_df=2, max_features=no_features, stop_words='english',strip_accents = 'unicode')
data_vectorized = ct_vectorizer.fit_transform(documents)
ct_feature_names = ct_vectorizer.get_feature_names()

num_topics = 10
# Run LDA
lda_model = LatentDirichletAllocation(n_components=num_topics, max_iter=10, 
                                learning_method='online', 
                                random_state=0,
                                evaluate_every = -1,       # compute perplexity every n iters, default: Don't
                                n_jobs = -1,               # Use all available CPUs
                               )
lda_output = lda_model.fit_transform(data_vectorized)

print("=====LDA=====")
display_topics(lda_model, ct_feature_names, num_top_words)



# # NMF is able to use tf-idf
# tfidf_vectorizer = TfidfVectorizer(max_df=0.95, min_df=2, max_features=no_features, stop_words='english')
# tfidf = tfidf_vectorizer.fit_transform(documents)
# tfidf_feature_names = tfidf_vectorizer.get_feature_names()

# num_top_words = 10

# # Run NMF
# nmf = NMF(n_components=num_topics, random_state=1, alpha=.1, l1_ratio=.5, init='nndsvd').fit(tfidf)
# LDA can only use raw term counts for LDA because it is a probabilistic graphical model

# print("=====NMF=====")
# display_topics(nmf, tfidf_feature_names, num_top_words)

In [20]:
'''
non-merged version 1 - gram 
Best Model's Params:  {'learning_decay': 0.7, 'n_components': 5}
Best Log Likelihood Score:  -971217.1918830868
Model Perplexity:  444.18687560698623

merged version 1 - gram 
Best Model's Params:  {'learning_decay': 0.7, 'n_components': 5}
Best Log Likelihood Score:  -961311.6105043223
Model Perplexity:  405.8577617448416
'''
from sklearn.model_selection import GridSearchCV
# Define Search Param
search_params = {'n_components': [2, 3, 4, 5, 6, 7], 'learning_decay': [.7]}

# Init the Model
lda = LatentDirichletAllocation()

# Init Grid Search Class
model = GridSearchCV(lda, param_grid=search_params, n_jobs=-1)

# Do the Grid Search
model.fit(data_vectorized)

# Best Model
best_lda_model = model.best_estimator_

# Model Parameters
print("Best Model's Params: ", model.best_params_)

# Log Likelihood Score
print("Best Log Likelihood Score: ", model.best_score_)

# Perplexity
print("Model Perplexity: ", best_lda_model.perplexity(data_vectorized))

/Users/jianhenghou/anaconda2/envs/py36/lib/python3.6/site-packages/sklearn/model_selection/_split.py:2053: FutureWarning: You should specify a value for 'cv' instead of relying on the default value. The default value will change from 3 to 5 in version 0.22.
  warnings.warn(CV_WARNING, FutureWarning)


Best Model's Params:  {'learning_decay': 0.7, 'n_components': 2}
Best Log Likelihood Score:  -282061.00252631115
Model Perplexity:  1185.0450471376164


In [21]:
# Show top n keywords for each topic
import numpy as np
def show_topics(vectorizer, lda_model, n_words):
    keywords = np.array(vectorizer.get_feature_names())
    topic_keywords = []
    for topic_weights in lda_model.components_:
        top_keyword_locs = (-topic_weights).argsort()[:n_words]
        topic_keywords.append(keywords.take(top_keyword_locs))
    return topic_keywords

topic_keywords = show_topics(vectorizer=ct_vectorizer, lda_model=best_lda_model, n_words=10)        



# Citalopram

## 1-gram Topic keywords

In [483]:
# Topic - Keywords Dataframe
df_topic_keywords = pd.DataFrame(topic_keywords)
df_topic_keywords.columns = ['Word '+str(i) for i in range(df_topic_keywords.shape[1])]
df_topic_keywords.index = ['Topic '+str(i) for i in range(df_topic_keywords.shape[0])]
df_topic_keywords

,Word 0,Word 1,Word 2,Word 3,Word 4,Word 5,Word 6,Word 7,Word 8,Word 9
Topic 0,work,anxiety,effect,things,think,cit,life,bite,way,try
Topic 1,anxiety,effect,citalopram,meds,feel,doctor,dose,depression,symptoms,medication


In [484]:
# 1-gram (replies merged to discussion)
LDAvis_1_gram = sklearn_lda.prepare(best_lda_model, data_vectorized, ct_vectorizer, mds='tsne')
LDAvis_1_gram

/Users/jianhenghou/anaconda2/envs/py36/lib/python3.6/site-packages/pyLDAvis/_prepare.py:257: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  return pd.concat([default_term_info] + list(topic_dfs))


PreparedData(topic_coordinates=               x           y  topics  cluster       Freq
topic                                                   
1     -60.312317  141.369629  1       1        53.235086
0      60.312561 -141.369141  2       1        46.764914, topic_info=     Category          Freq        Term         Total  loglift  logprob
661   Default  4086.000000   feel        4086.000000   30.0000  30.0000
1791  Default  2483.000000   think       2483.000000   29.0000  29.0000
1099  Default  4818.000000   meds        4818.000000   28.0000  28.0000
1198  Default  1451.000000   need        1451.000000   27.0000  27.0000
266   Default  1629.000000   care        1629.000000   26.0000  26.0000
1675  Default  1672.000000   start       1672.000000   25.0000  25.0000
319   Default  6639.000000   citalopram  6639.000000   24.0000  24.0000
1951  Default  1318.000000   withdrawal  1318.000000   23.0000  23.0000
1964  Default  5525.000000   work        5525.000000   22.0000  22.0000
1747  Default  2322.000000   symptoms    2322.000000   21.0000  21.0000
74    Default  10619.000000  anxiety     10619.000000  20.0000  20.0000
1035  Default  750.000000    lol         750.000000    19.0000  19.0000
446   Default  688.000000    david       688.000000    18.0000  18.0000
1851  Default  1986.000000   try         1986.000000   17.0000  17.0000
535   Default  4431.000000   dose        4431.000000   16.0000  16.0000
576   Default  9633.000000   effect      9633.000000   15.0000  15.0000
276   Default  648.000000    celexa      648.000000    14.0000  14.0000
1732  Default  1140.000000   support     1140.000000   13.0000  13.0000
631   Default  1712.000000   experience  1712.000000   12.0000  12.0000
1790  Default  3920.000000   things      3920.000000   11.0000  11.0000
314   Default  3491.000000   cit         3491.000000   10.0000  10.0000
526   Default  4659.000000   doctor      4659.000000   9.0000   9.0000 
1665  Default  820.000000    ssri        820.000000    8.0000   8.0000 
824   Default  856.000000    home        856.000000    7.0000   7.0000 
964   Default  441.000000    kid         441.000000    6.0000   6.0000 
551   Default  463.000000    drink       463.000000    5.0000   5.0000 
1094  Default  2492.000000   medication  2492.000000   4.0000   4.0000 
1285  Default  410.000000    partner     410.000000    3.0000   3.0000 
1456  Default  917.000000    regard      917.000000    2.0000   2.0000 
936   Default  892.000000    job         892.000000    1.0000   1.0000 
...       ...         ...    ...                ...       ...      ... 
1851  Topic2   1526.475689   try         1986.693463   0.4965  -4.6522 
1456  Topic2   753.782823    regard      917.261170    0.5637  -5.3578 
1964  Topic2   3653.334187   work        5525.459850   0.3463  -3.7796 
936   Topic2   731.124790    job         892.419197    0.5607  -5.3884 
1757  Topic2   618.715608    talk        742.345461    0.5779  -5.5553 
1790  Topic2   2574.647413   things      3920.223925   0.3396  -4.1295 
314   Topic2   2311.202824   cit         3491.801494   0.3474  -4.2374 
703   Topic2   875.585114    forum       1176.743833   0.4644  -5.2081 
172   Topic2   1547.038451   bite        2360.686394   0.3374  -4.6388 
1011  Topic2   1991.291065   life        3281.289065   0.2606  -4.3864 
1789  Topic2   1500.141340   thing       2446.286899   0.2710  -4.6696 
650   Topic2   801.939908    family      1125.103140   0.4214  -5.2959 
1819  Topic2   571.793418    tomorrow    730.462926    0.5151  -5.6342 
1920  Topic2   1538.292637   way         2999.310059   0.0923  -4.6445 
1986  Topic2   705.893906    yesterday   1010.820621   0.4010  -5.4235 
149   Topic2   732.577161    bed         1070.473328   0.3807  -5.3864 
596   Topic2   773.582970    end         1170.893234   0.3455  -5.3319 
74    Topic2   3100.041603   anxiety     10619.091521 -0.4712  -3.9438 
576   Topic2   2836.399486   effect      9633.623214  -0.4627  -4.0327 
1044  Topic2   1445.080224   lot         3266.004014  -0.

##  2-gram Topic keywords¶

In [488]:
# Topic - Keywords Dataframe
df_topic_keywords = pd.DataFrame(topic_keywords)
df_topic_keywords.columns = ['Word '+str(i) for i in range(df_topic_keywords.shape[1])]
df_topic_keywords.index = ['Topic '+str(i) for i in range(df_topic_keywords.shape[0])]
df_topic_keywords

,Word 0,Word 1,Word 2,Word 3,Word 4,Word 5,Word 6,Word 7,Word 8,Word 9
Topic 0,side_effect,take_care,panic_attack,regard_david,take_cit,take_citalopram,dry_mouth,keep_post,increase_anxiety,one_thing
Topic 1,panic_attack,take_citalopram,side_effect,withdrawal_symptoms,cold_turkey,take_care,brain_zap,blood_pressure,mental_health,long_term


In [489]:
# 2-gram (replies merged to discussion)
LDAvis_2_gram = sklearn_lda.prepare(best_lda_model, data_vectorized, ct_vectorizer, mds='tsne')
LDAvis_2_gram

/Users/jianhenghou/anaconda2/envs/py36/lib/python3.6/site-packages/pyLDAvis/_prepare.py:257: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  return pd.concat([default_term_info] + list(topic_dfs))


PreparedData(topic_coordinates=               x           y  topics  cluster       Freq
topic                                                   
0     -60.312317  141.369629  1       1        64.608412
1      60.312561 -141.369141  2       1        35.391588, topic_info=     Category         Freq                 Term        Total  loglift  logprob
1597  Default  7809.000000  side_effect          7809.000000  30.0000  30.0000
1944  Default  449.000000   withdrawal_symptoms  449.000000   29.0000  29.0000
385   Default  377.000000   cold_turkey          377.000000   28.0000  28.0000
1777  Default  708.000000   take_citalopram      708.000000   27.0000  27.0000
309   Default  211.000000   brain_zap            211.000000   26.0000  26.0000
1478  Default  427.000000   regard_david         427.000000   25.0000  25.0000
1357  Default  1168.000000  panic_attack         1168.000000  24.0000  24.0000
290   Default  172.000000   blood_pressure       172.000000   23.0000  23.0000
1378  Default  142.000000   pill_cutter          142.000000   22.0000  22.0000
1581  Default  115.000000   sex_drive            115.000000   21.0000  21.0000
547   Default  125.000000   else_experience      125.000000   20.0000  20.0000
1256  Default  105.000000   nervous_system       105.000000   19.0000  19.0000
1179  Default  220.000000   mental_health        220.000000   18.0000  18.0000
1596  Default  129.000000   side_affect          129.000000   17.0000  17.0000
1092  Default  201.000000   long_term            201.000000   16.0000  16.0000
971   Default  223.000000   keep_post            223.000000   15.0000  15.0000
292   Default  89.000000    blood_test           89.000000    14.0000  14.0000
1796  Default  177.000000   take_mine            177.000000   13.0000  13.0000
1383  Default  72.000000    please_help          72.000000    12.0000  12.0000
1702  Default  162.000000   stick_plaster        162.000000   11.0000  11.0000
691   Default  55.000000    gain_weight          55.000000    10.0000  10.0000
1773  Default  61.000000    take_celexa          61.000000    9.0000   9.0000 
1103  Default  51.000000    lose_weight          51.000000    8.0000   8.0000 
499   Default  128.000000   dose_increase        128.000000   7.0000   7.0000 
1599  Default  69.000000    similar_experience   69.000000    6.0000   6.0000 
1719  Default  49.000000    stop_citalopram      49.000000    5.0000   5.0000 
131   Default  144.000000   anti_depressants     144.000000   4.0000   4.0000 
1940  Default  163.000000   withdrawal_effect    163.000000   3.0000   3.0000 
938   Default  208.000000   increase_anxiety     208.000000   2.0000   2.0000 
176   Default  51.000000    anyone_experience    51.000000    1.0000   1.0000 
...       ...        ...                  ...          ...       ...      ... 
1054  Topic2   32.885738    like_symptoms        35.032875    0.9754  -6.4997 
1826  Topic2   29.504556    term_memory          31.142479    0.9847  -6.6082 
1596  Topic2   105.245643   side_affect          129.299172   0.8329  -5.3365 
1777  Topic2   474.104617   take_citalopram      708.872432   0.6364  -3.8313 
1599  Topic2   60.052939    similar_experience   69.054810    0.8990  -5.8975 
859   Topic2   45.031640    heart_beat           50.583609    0.9224  -6.1854 
1048  Topic2   38.213709    like_im              41.836432    0.9481  -6.3496 
1704  Topic2   45.668662    still_feel           52.520518    0.8989  -6.1713 
1092  Topic2   134.529310   long_term            201.009751   0.6371  -5.0910 
1357  Topic2   547.353636   panic_attack         1168.274268  0.2805  -3.6877 
1179  Topic2   143.626772   mental_health        220.400296   0.6105  -5.0255 
1038  Topic2   52.198799    light_head           64.163276    0.8323  -6.0377 
131   Topic2   91.186216    anti_depressants     144.643736   0.5773  -5.4798 
1940  Topic2   97.803243    withdrawal_effect    163.048817   0.5276  -5.4098 
1752  Topic2   63.727550    suicidal_thoughts    95.164331    0.6377  -5.8381 
869   Topic2   5

# Abdominal Disorders

## 1-gram Topic keywords

In [465]:
# Topic - Keywords Dataframe
df_topic_keywords = pd.DataFrame(topic_keywords)
df_topic_keywords.columns = ['Word '+str(i) for i in range(df_topic_keywords.shape[1])]
df_topic_keywords.index = ['Topic '+str(i) for i in range(df_topic_keywords.shape[0])]
df_topic_keywords

,Word 0,Word 1,Word 2,Word 3,Word 4,Word 5,Word 6,Word 7,Word 8,Word 9
Topic 0,pain,stomach,food,doctor,eat,symptoms,foods,lot,help,feel
Topic 1,pain,test,doctor,blood,symptoms,stomach,endoscopy,stool,problems,gastritis


In [466]:
# 1-gram (replies merged to discussion)
LDAvis_1_gram = sklearn_lda.prepare(best_lda_model, data_vectorized, ct_vectorizer, mds='tsne')
LDAvis_1_gram

/Users/jianhenghou/anaconda2/envs/py36/lib/python3.6/site-packages/pyLDAvis/_prepare.py:257: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  return pd.concat([default_term_info] + list(topic_dfs))


PreparedData(topic_coordinates=               x           y  topics  cluster       Freq
topic                                                   
1     -60.312317  141.369629  1       1        57.001757
0      60.312561 -141.369141  2       1        42.998243, topic_info=     Category          Freq          Term         Total  loglift  logprob
1781  Default  7082.000000   test          7082.000000   30.0000  30.0000
709   Default  3305.000000   food          3305.000000   29.0000  29.0000
574   Default  2315.000000   eat           2315.000000   28.0000  28.0000
710   Default  1690.000000   foods         1690.000000   27.0000  27.0000
203   Default  3887.000000   blood         3887.000000   26.0000  26.0000
1935  Default  1406.000000   water         1406.000000   25.0000  25.0000
1684  Default  7055.000000   stomach       7055.000000   24.0000  24.0000
746   Default  1084.000000   gas           1084.000000   23.0000  23.0000
667   Default  1584.000000   feel          1584.000000   22.0000  22.0000
599   Default  1619.000000   endoscopy     1619.000000   21.0000  21.0000
1129  Default  640.000000    milk          640.000000    20.0000  20.0000
1855  Default  1022.000000   try           1022.000000   19.0000  19.0000
554   Default  677.000000    drink         677.000000    18.0000  18.0000
505   Default  1185.000000   diet          1185.000000   17.0000  17.0000
274   Default  1023.000000   cancer        1023.000000   16.0000  16.0000
1263  Default  12389.000000  pain          12389.000000  15.0000  15.0000
727   Default  485.000000    fruit         485.000000    14.0000  14.0000
882   Default  1577.000000   ibs           1577.000000   13.0000  13.0000
198   Default  1195.000000   bloat         1195.000000   12.0000  12.0000
1432  Default  841.000000    pylori        841.000000    11.0000  11.0000
1507  Default  1289.000000   result        1289.000000   10.0000  10.0000
1056  Default  2541.000000   lot           2541.000000   9.0000   9.0000 
1762  Default  414.000000    tea           414.000000    8.0000   8.0000 
395   Default  1011.000000   constipation  1011.000000   7.0000   7.0000 
457   Default  433.000000    dairy         433.000000    6.0000   6.0000 
1089  Default  420.000000    meals         420.000000    5.0000   5.0000 
1514  Default  248.000000    rice          248.000000    4.0000   4.0000 
348   Default  332.000000    coffee        332.000000    3.0000   3.0000 
1309  Default  441.000000    period        441.000000    2.0000   2.0000 
443   Default  573.000000    ct            573.000000    1.0000   1.0000 
...       ...         ...    ..                   ...       ...      ... 
767   Topic2   157.554399    ginger        168.202518    0.7786  -6.7350 
153   Topic2   261.862562    bathroom      294.967443    0.7250  -6.2269 
505   Topic2   909.786520    diet          1185.992592   0.5789  -4.9815 
1309  Topic2   372.561898    period        441.621241    0.6740  -5.8743 
1718  Topic2   354.621026    sugar         419.833626    0.6752  -5.9237 
1684  Topic2   4079.229995   stomach       7055.563080   0.2961  -3.4811 
882   Topic2   1114.302317   ibs           1577.153896   0.4966  -4.7788 
198   Topic2   869.068307    bloat         1195.302768   0.5253  -5.0273 
1263  Topic2   5948.487322   pain          12389.473955  0.1103  -3.1039 
502   Topic2   318.741733    diary         373.267124    0.6861  -6.0304 
395   Topic2   719.012262    constipation  1011.084955   0.5031  -5.2169 
1056  Topic2   1485.345894   lot           2541.259631   0.3070  -4.4913 
833   Topic2   1302.277583   help          2339.367061   0.2582  -4.6229 
539   Topic2   2524.024283   doctor        6448.226274  -0.0939  -3.9611 
1512  Topic2   691.820272    rib           1118.994678   0.3632  -5.2554 
221   Topic2   821.703512    bowel         1457.927162   0.2706  -5.0834 
1696  Topic2   589.732132    stress        918.427204    0.4010  -5.4151 
1174  Topic2   997.141155    nausea        1994.902960   0.1506  -4.8899 
2     Topic2   784.612197    

##  2-gram Topic keywords¶

In [470]:
# Topic - Keywords Dataframe
df_topic_keywords = pd.DataFrame(topic_keywords)
df_topic_keywords.columns = ['Word '+str(i) for i in range(df_topic_keywords.shape[1])]
df_topic_keywords.index = ['Topic '+str(i) for i in range(df_topic_keywords.shape[0])]
df_topic_keywords

,Word 0,Word 1,Word 2,Word 3,Word 4,Word 5,Word 6,Word 7,Word 8,Word 9
Topic 0,acid_reflux,side_effect,h_pylori,_pylori,blood_test,breath_test,stomach_pain,stool_test,hiatus_hernia,stomach_acid
Topic 1,blood_test,abdominal_pain,right_side,leave_side,stomach_pain,back_pain,rib_cage,blood_work,stool_test,sharp_pain


In [471]:
# 2-gram (replies merged to discussion)
LDAvis_2_gram = sklearn_lda.prepare(best_lda_model, data_vectorized, ct_vectorizer, mds='tsne')
LDAvis_2_gram

/Users/jianhenghou/anaconda2/envs/py36/lib/python3.6/site-packages/pyLDAvis/_prepare.py:257: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  return pd.concat([default_term_info] + list(topic_dfs))


PreparedData(topic_coordinates=               x           y  topics  cluster       Freq
topic                                                   
1     -60.312317  141.369629  1       1        52.964989
0      60.312561 -141.369141  2       1        47.035011, topic_info=     Category         Freq                 Term        Total  loglift  logprob
1526  Default  733.000000   right_side           733.000000   30.0000  30.0000
75    Default  598.000000   acid_reflux          598.000000   29.0000  29.0000
54    Default  869.000000   abdominal_pain       869.000000   28.0000  28.0000
1058  Default  533.000000   leave_side           533.000000   27.0000  27.0000
273   Default  1391.000000  blood_test           1391.000000  26.0000  26.0000
178   Default  340.000000   back_pain            340.000000   25.0000  25.0000
1609  Default  444.000000   side_effect          444.000000   24.0000  24.0000
35    Default  305.000000   _pylori              305.000000   23.0000  23.0000
304   Default  262.000000   breath_test          262.000000   22.0000  22.0000
1507  Default  265.000000   rib_cage             265.000000   21.0000  21.0000
1691  Default  175.000000   stomach_acid         175.000000   20.0000  20.0000
925   Default  188.000000   hiatus_hernia        188.000000   19.0000  19.0000
212   Default  184.000000   belly_button         184.000000   18.0000  18.0000
1523  Default  179.000000   right_rib            179.000000   17.0000  17.0000
1710  Default  156.000000   stomach_line         156.000000   16.0000  16.0000
1592  Default  200.000000   sharp_pain           200.000000   15.0000  15.0000
1146  Default  171.000000   lower_abdomen        171.000000   14.0000  14.0000
923   Default  172.000000   hiatal_hernia        172.000000   13.0000  13.0000
289   Default  153.000000   bowel_habit          153.000000   12.0000  12.0000
1918  Default  159.000000   urine_test           159.000000   11.0000  11.0000
1122  Default  137.000000   long_term            137.000000   10.0000  10.0000
382   Default  128.000000   chronic_gastritis    128.000000   9.0000   9.0000 
92    Default  113.000000   aloe_vera            113.000000   8.0000   8.0000 
266   Default  169.000000   blood_pressure       169.000000   7.0000   7.0000 
1383  Default  116.000000   panic_attack         116.000000   6.0000   6.0000 
1186  Default  97.000000    mastic_gum           97.000000    5.0000   5.0000 
1023  Default  118.000000   kidney_stone         118.000000   4.0000   4.0000 
277   Default  275.000000   blood_work           275.000000   3.0000   3.0000 
881   Default  429.000000   h_pylori             429.000000   2.0000   2.0000 
519   Default  142.000000   drink_water          142.000000   1.0000   1.0000 
...       ...         ...           ...                 ...      ...      ... 
35    Topic2   283.752013   _pylori              305.990119   0.6788  -4.6176 
1697  Topic2   85.602817    stomach_cancer       89.026088    0.7151  -5.8159 
1609  Topic2   399.381745   side_effect          444.539540   0.6472  -4.2757 
917   Topic2   59.349549    helicobacter_pylori  61.635222    0.7165  -6.1822 
1792  Topic2   44.750012    take_omeprazole      45.991601    0.7269  -6.4646 
205   Topic2   62.276690    bake_soda            65.531211    0.7033  -6.1341 
465   Topic2   49.884245    diet_change          51.850930    0.7156  -6.3560 
519   Topic2   126.721398   drink_water          142.613907   0.6361  -5.4237 
571   Topic2   74.433380    empty_study          80.165522    0.6801  -5.9558 
1679  Topic2   69.853705    spicy_foods          75.276533    0.6795  -6.0193 
1801  Topic2   107.293153   take_probiotics      122.076158   0.6252  -5.5901 
404   Topic2   95.136576    colon_cancer         107.441849   0.6326  -5.7103 
558   Topic2   134.219392   eat_something        159.069864   0.5844  -5.3662 
881   Topic2   306.689739   h_pylori             429.202072   0.4182  -4.5398 
1715  Topic2   158.119554   stomach_problems     198.877804   0.5249  -5.2023 
1709  Topic2   1

# Menopause

## 1-gram Topic keywords

In [447]:
# Topic - Keywords Dataframe
df_topic_keywords = pd.DataFrame(topic_keywords)
df_topic_keywords.columns = ['Word '+str(i) for i in range(df_topic_keywords.shape[1])]
df_topic_keywords.index = ['Topic '+str(i) for i in range(df_topic_keywords.shape[0])]
df_topic_keywords

,Word 0,Word 1,Word 2,Word 3,Word 4,Word 5,Word 6,Word 7,Word 8,Word 9
Topic 0,life,things,symptoms,menopause,way,work,women,husband,anxiety,lot
Topic 1,eye,symptoms,head,hair,skin,menopause,burn,mouth,migraines,headaches
Topic 2,anxiety,feel,symptoms,heart,menopause,doctor,period,blood,peri,hormones
Topic 3,hrt,doctor,period,menopause,progesterone,periods,symptoms,blood,test,estrogen
Topic 4,pain,magnesium,help,stomach,weight,symptoms,shoulder,ache,lot,period


In [448]:
# 1-gram (replies merged to discussion)
LDAvis_1_gram = sklearn_lda.prepare(best_lda_model, data_vectorized, ct_vectorizer, mds='tsne')
LDAvis_1_gram

/Users/jianhenghou/anaconda2/envs/py36/lib/python3.6/site-packages/pyLDAvis/_prepare.py:257: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  return pd.concat([default_term_info] + list(topic_dfs))


PreparedData(topic_coordinates=                x           y  topics  cluster       Freq
topic                                                    
3      54.277729  -162.572998  1       1        28.608668
0      154.828629 -116.519310  2       1        22.937315
2     -14.808171  -30.409092   3       1        20.959202
4      94.882797  -19.064495   4       1        16.633582
1     -53.568260  -138.058838  5       1        10.861233, topic_info=     Category          Freq          Term         Total  loglift  logprob
1261  Default  9464.000000   pain          9464.000000   30.0000  30.0000
649   Default  6313.000000   feel          6313.000000   29.0000  29.0000
69    Default  8609.000000   anxiety       8609.000000   28.0000  28.0000
627   Default  2142.000000   eye           2142.000000   27.0000  27.0000
786   Default  1820.000000   hair          1820.000000   26.0000  26.0000
802   Default  3191.000000   head          3191.000000   25.0000  25.0000
808   Default  3691.000000   heart         3691.000000   24.0000  24.0000
1387  Default  3139.000000   progesterone  3139.000000   23.0000  23.0000
1052  Default  2035.000000   magnesium     2035.000000   22.0000  22.0000
180   Default  2465.000000   bleed         2465.000000   21.0000  21.0000
1722  Default  13875.000000  symptoms      13875.000000  20.0000  20.0000
1593  Default  1401.000000   skin          1401.000000   19.0000  19.0000
1663  Default  2109.000000   stomach       2109.000000   18.0000  18.0000
244   Default  1170.000000   burn          1170.000000   17.0000  17.0000
1573  Default  1482.000000   shoulder      1482.000000   16.0000  16.0000
1008  Default  5155.000000   life          5155.000000   15.0000  15.0000
1930  Default  2077.000000   weight        2077.000000   14.0000  14.0000
603   Default  2970.000000   estrogen      2970.000000   13.0000  13.0000
1187  Default  1438.000000   neck          1438.000000   12.0000  12.0000
1117  Default  1080.000000   migraines     1080.000000   11.0000  11.0000
1164  Default  1044.000000   mouth         1044.000000   10.0000  10.0000
847   Default  6754.000000   hrt           6754.000000   9.0000   9.0000 
804   Default  1486.000000   headaches     1486.000000   8.0000   8.0000 
947   Default  1009.000000   joint         1009.000000   7.0000   7.0000 
104   Default  1753.000000   attack        1753.000000   6.0000   6.0000 
1265  Default  1506.000000   palpitations  1506.000000   5.0000   5.0000 
124   Default  1336.000000   b6            1336.000000   4.0000   4.0000 
858   Default  2154.000000   husband       2154.000000   3.0000   3.0000 
227   Default  1811.000000   breast        1811.000000   2.0000   2.0000 
1116  Default  820.000000    migraine      820.000000    1.0000   1.0000 
...       ...         ...         ...             ...       ...      ... 
1192  Topic5   154.733957    neurologist   182.531238    2.0548  -6.1978 
1538  Topic5   557.571860    sensation     827.170906    1.8256  -4.9160 
803   Topic5   574.731042    headache      895.555109    1.7764  -4.8856 
1794  Topic5   184.852873    tinnitus      231.688563    1.9941  -6.0200 
1204  Topic5   131.870729    nose          154.505763    2.0616  -6.3577 
802   Topic5   1526.665079   head          3191.326865   1.4826  -3.9087 
628   Topic5   636.313573    face          1090.369958   1.6814  -4.7839 
804   Topic5   781.615872    headaches     1486.028283   1.5775  -4.5782 
650   Topic5   563.896811    feet          979.737398    1.6676  -4.9047 
1187  Topic5   741.975467    neck          1438.336796   1.5580  -4.6302 
997   Topic5   540.993808    legs          984.460788    1.6213  -4.9461 
46    Topic5   250.508047    allergies     365.286676    1.8428  -5.7161 
1776  Topic5   415.302683    throat        940.229459    1.4029  -5.2105 
1371  Topic5   572.545166    pressure      1729.020005   1.1148  -4.8895 
1722  Topic5   1671.393168   symptoms      13875.015784  0.1035  -3.8181 
789   Topic5   469.634730    hand          1336.802924   1.1739  -5.0876 

##  2-gram Topic keywords¶

In [22]:
# Topic - Keywords Dataframe
df_topic_keywords = pd.DataFrame(topic_keywords)
df_topic_keywords.columns = ['Word '+str(i) for i in range(df_topic_keywords.shape[1])]
df_topic_keywords.index = ['Topic '+str(i) for i in range(df_topic_keywords.shape[0])]
df_topic_keywords

,Word 0,Word 1,Word 2,Word 3,Word 4,Word 5,Word 6,Word 7,Word 8,Word 9
Topic 0,blood_test,hot_flush,side_effect,take_hrt,take_care,blood_pressure,hot_flash,breast_cancer,weight_gain,last_period
Topic 1,hot_flash,panic_attack,take_care,joint_pain,side_effect,blood_test,hot_flush,health_anxiety,peri_menopause,back_pain


In [23]:
# 2-gram (replies merged to discussion)
LDAvis_2_gram = sklearn_lda.prepare(best_lda_model, data_vectorized, ct_vectorizer, mds='tsne')
LDAvis_2_gram

/Users/jianhenghou/anaconda2/envs/py36/lib/python3.6/site-packages/pyLDAvis/_prepare.py:257: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  return pd.concat([default_term_info] + list(topic_dfs))


PreparedData(topic_coordinates=               x           y  topics  cluster       Freq
topic                                                   
0     -60.312317  141.369629  1       1        50.282544
1      60.312561 -141.369141  2       1        49.717456, topic_info=     Category         Freq                 Term        Total  loglift  logprob
909   Default  745.000000   joint_pain           745.000000   30.0000  30.0000
1762  Default  694.000000   take_hrt             694.000000   29.0000  29.0000
1355  Default  892.000000   panic_attack         892.000000   28.0000  28.0000
849   Default  1825.000000  hot_flash            1825.000000  27.0000  27.0000
973   Default  361.000000   leave_side           361.000000   26.0000  26.0000
158   Default  452.000000   back_pain            452.000000   25.0000  25.0000
263   Default  460.000000   breast_cancer        460.000000   24.0000  24.0000
739   Default  572.000000   health_anxiety       572.000000   23.0000  23.0000
432   Default  268.000000   dry_eye              268.000000   22.0000  22.0000
1513  Default  247.000000   right_side           247.000000   21.0000  21.0000
773   Default  267.000000   heavy_bleed          267.000000   20.0000  20.0000
313   Default  243.000000   chest_pain           243.000000   19.0000  19.0000
1421  Default  418.000000   post_meno            418.000000   18.0000  18.0000
30    Default  214.000000   _c                   214.000000   17.0000  17.0000
1233  Default  212.000000   neck_pain            212.000000   16.0000  16.0000
1185  Default  313.000000   mood_swing           313.000000   15.0000  15.0000
448   Default  241.000000   early_peri           241.000000   14.0000  14.0000
277   Default  201.000000   burn_sensation       201.000000   13.0000  13.0000
737   Default  197.000000   head_pressure        197.000000   12.0000  12.0000
1927  Default  197.000000   vitamin_b6           197.000000   11.0000  11.0000
435   Default  290.000000   dry_mouth            290.000000   10.0000  10.0000
603   Default  199.000000   freeze_shoulder      199.000000   9.0000   9.0000 
1173  Default  192.000000   mirena_coil          192.000000   8.0000   8.0000 
61    Default  301.000000   acid_reflux          301.000000   7.0000   7.0000 
1586  Default  170.000000   shoulder_pain        170.000000   6.0000   6.0000 
266   Default  205.000000   breast_pain          205.000000   5.0000   5.0000 
828   Default  244.000000   hormone_replacement  244.000000   4.0000   4.0000 
1961  Default  160.000000   whole_body           160.000000   3.0000   3.0000 
325   Default  167.000000   coconut_oil          167.000000   2.0000   2.0000 
529   Default  159.000000   family_history       159.000000   1.0000   1.0000 
...       ...         ...              ...              ...      ...      ... 
621   Topic2   133.678161   gain_weight          136.771759   0.6759  -6.0701 
266   Topic2   199.019900   breast_pain          205.698823   0.6658  -5.6721 
158   Topic2   430.540695   back_pain            452.346714   0.6494  -4.9005 
1011  Topic2   131.362926   like_something       134.843677   0.6727  -6.0876 
739   Topic2   518.912444   health_anxiety       572.463035   0.6006  -4.7138 
435   Topic2   270.242583   dry_mouth            290.410772   0.6268  -5.3662 
1651  Topic2   119.120119   sore_throat          123.228204   0.6649  -6.1854 
1355  Topic2   778.550130   panic_attack         892.089875   0.5627  -4.3081 
1259  Topic2   135.512571   new_symptom          141.704632   0.6541  -6.0565 
1320  Topic2   142.315735   one_side             149.487000   0.6497  -6.0075 
61    Topic2   274.597451   acid_reflux          301.253998   0.6062  -5.3502 
1210  Topic2   176.352454   muscle_pain          189.395272   0.6275  -5.7931 
849   Topic2   1384.833607  hot_flash            1825.978928  0.4223  -3.7322 
1693  Topic2   238.279171   stomach_issue        270.384010   0.5724  -5.4921 
1042  Topic2   247.740777   lose_weight          298.031214   0.5140  -5.4532 
407   Topic2   1